In [107]:
#!/usr/bin/env python
import snowflake.connector
import time
import hashlib as hashlib
import pandas as pd
import numpy as np
import calendar
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from hashlib import md5
###
# https://docs.snowflake.com/en/user-guide/python-connector-example.html
###
from datetime import date, datetime, timedelta
from dateutil.parser import parse as dateParse

In [109]:
#---------------------------------------
# CONNECT TO SNOWFLAKE
#---------------------------------------

# Create main connection context
cxn = snowflake.connector.connect(
    user='snowflakemeetup20210524',
    password='...',
    account='xxx3493'
    
    #extras 
    ,
    warehouse='DATALAKEHOUSE_WH',
    database='STARTSCHEMA_COVID19', #DEMO_DB
    schema='PUBLIC'
#     , protocol='https'
#     , port=port
#     ,
#     session_parameters={
#     'QUERY_TAG': 'EndOfMonthFinancials',
#     }
    )
# -- End Connection Creation

# CREATE USER snowflakemeetup20210524 PASSWORD = '*************' 
# COMMENT = 'Delete After the Event' LOGIN_NAME = 'snowflakemeetup20210524' 
# DISPLAY_NAME = 'AICG - We Love Data' DEFAULT_ROLE = "DATALAKEHOUSE_ROLE" DEFAULT_WAREHOUSE = 'FIVETRAN_WAREHOUSE' 
# MUST_CHANGE_PASSWORD = FALSE;
# GRANT ROLE "DATALAKEHOUSE_ROLE" TO USER snowflakemeetup20210524;


In [110]:
#---------------------------------------
# GET CURRENT VERSION OF SNOWFLAKE
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()
try:
    # Gets the version
    cxn_cursor.execute("SELECT current_version()")
    
    # Gets the first column and record returned from the cursor and returns
    one_row = cxn_cursor.fetchone()
    print(one_row[0])
    
    #alternatively if returning two columns then assign in dict
    #col1, col2 = con.cursor().execute("SELECT col1, col2 FROM testtable").fetchone()
    #print('{0}, {1}'.format(col1, col2))
    
    
finally:
    cxn_cursor.close()
    

5.19.2


In [ ]:
#---------------------------------------
# Basic Retrieval from the COVID19 Dataset
# -- WHO Daily Report Data
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()
try:
    # Gets the data via SQL Select Query
    cxn_cursor.execute("SELECT COUNTRY_REGION, CASES_TOTAL, CASES AS CASES_TODAY FROM STARSCHEMA_COVID19.PUBLIC.WHO_DAILY_REPORT;")
    
    # Loops/iterates through the cursor
    for rec in cxn_cursor:
        print('%s, %s, %s' % (rec[0], rec[1], rec[2]))
        
    # Snowflake execute alternatively returns an iterator and can be called as the for loop
    #for (col1, col2) in con.cursor().execute("SELECT col1, col2 FROM testtable"):
    #    print('{0}, {1}'.format(col1, col2))
    
finally:
    cxn_cursor.close()

In [112]:
#---------------------------------------
# PANDAS DataFrame Retrieval from the COVID19 Dataset
# -- WHO Daily Report Data
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()

nData = df
try:
    # Gets the SQL via SELECT Query
    cxn_cursor.execute("SELECT COUNTRY_REGION, CASES_TOTAL, CASES AS CASES_TODAY FROM STARSCHEMA_COVID19.PUBLIC.WHO_DAILY_REPORT")

    # Loops/iterates through the cursor
#     rows = 0
#     while True:
#         data_output = cxn_cursor.fetchmany(5000)
#         if not data_output:
#             break
#         pdf = pd.DataFrame(data_output, columns=cxn_cursor.description)
#         rows += pdf.shape[0]
    #print(rows)
    pd.set_option("max_columns", 2)
    df = cxn_cursor.fetch_pandas_all()
    
#     print(df)
    
finally:
    cxn_cursor.close()
    
df.head(5)

,COUNTRY_REGION,...,CASES_TODAY
0,Global,...,563692
1,United States,...,27921
2,India,...,240842
3,Brazil,...,76855
4,France,...,0


In [113]:
#---------------------------------------
# CREATE A SCHEMA
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()
try:
    # Switch Context
    cxn_cursor.execute("USE DATABASE DEMO_DB")
    cxn_cursor.execute("CREATE SCHEMA IF NOT EXISTS MEETUP_DB_20210524")
    cxn_cursor.execute("USE SCHEMA MEETUP_DB_20210524")
    

    # Gets the version
    cxn_cursor.execute("SELECT current_version()")
    
    # Gets the first record returned from the cursor and returns
    one_row = cxn_cursor.fetchone()
    print(one_row[0])
    
finally:
    cxn_cursor.close()

5.19.2


In [114]:
#---------------------------------------
# CREATE A TABLE AND INSERT SOME RECORDS
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()
try:
    # Creates the Table
    cxn_cursor.execute("CREATE OR REPLACE TABLE MEETUP_GUESTS (guest_name string, guest_phone varchar(12), guest_interests varchar(1000))")
    
    # Insert a record into the table to confirm table exists
    cxn_cursor.execute("""
        INSERT INTO MEETUP_GUESTS (guest_name, guest_phone, guest_interests) 
        VALUES ('Elon Musk', '888-SKY-HIGH', 'Rockets, Cars, Space, Cryto, DataLakeHouse.io')
    """ 
    )
    
    # Just a validation to make sure the script didn't abend/crash before here
    cxn_cursor.execute("SELECT current_version()")
    
    # Get the records from the table
    cxn_cursor.execute("SELECT * FROM MEETUP_GUESTS")
    
    print("Here is the Snowflake Query ID that handled the above cursor execution")
    print(cxn_cursor.sfqid)
    
    print("")

    # Print
    df = cxn_cursor.fetch_pandas_all()
    
    print(df.head(100))
    
finally:
    cxn_cursor.close()

Here is the Snowflake Query ID that handled the above cursor execution
019c7833-0400-6ab5-000b-3583000dc182

  GUEST_NAME  ...                                GUEST_INTERESTS
0  Elon Musk  ...  Rockets, Cars, Space, Cryto, DataLakeHouse.io

[1 rows x 3 columns]


In [118]:
#---------------------------------------
# INSERT RECORDS WITH A EXECUTEMANY()
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()

nData = df

try:
    # New DataFrame with More Guests Names

    # dictionary with list object in values
    #df = pd.DataFrame([('Albert', 'E', 'Theories, Collectibles, Science'), ('Nikoli', 'Tesla', 'Eletricity, Winning, Soft Noises, Ideas')], columns=['guest_name', 'guest_phone', 'guest_interests'])
    
#     rows_to_insert = [('Albert E.', '888-ALBERTE', 'Theories, Collectibles, Science'), ('Nikoli Tesla', '555-RDE-LTNG', 'Eletricity, Winning, Soft Noises, Ideas')]
#     cxn_cursor.execute(
#         "insert into MEETUP_GUESTS (guest_name, guest_phone, guest_interests) values (?, ?, ?)",
#         rows_to_insert)

    #INSERT PLAINLY One LINE AT A TIME
    cxn_cursor.execute("INSERT INTO MEETUP_GUESTS VALUES('Albert E.', '888-ALBERTE', 'Theories, Collectibles, Science')")
    cxn_cursor.execute("INSERT INTO MEETUP_GUESTS VALUES('Nikoli Tesla', '555-RDE-LTNG', 'Eletricity, Winning, Soft Noises, Ideas')")

    
    print("")

    # Get the records from the table
    nData = cxn_cursor.execute("SELECT * FROM MEETUP_GUESTS").fetch_pandas_all()
    
    print("Here is the Snowflake Query ID that handled the above cursor execution")
    print(cxn_cursor.sfqid)
    
    print("")
    
    
except snowflake.connector.errors.ProgrammingError as e:
    # default error message
    print(e)
    # customer error message
    print('Error {0} ({1}): {2} ({3})'.format(e.errno, e.sqlstate, e.msg, e.sfqid))
    
    # ROLLBACK any part of this transaction
    cxn_cursor.rollback()
    raise e
finally:
    cxn_cursor.close()
    
nData.head(5)


DatabaseError: 250002 (08003): Connection is closed

In [ ]:
#---------------------------------------
# LOAD DATA FROM FILES ON SERVER OR LAPTOP
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()

nData = df

try:

    # Use the PUT Operation into an EXISTING STAGE by the name of, 
    #cxn_cursor.execute("PUT file:\/Users\/christian\/Coding\/tmp\/meetup_guest2.csv @default_loads")
#     cxn_cursor.execute("COPY INTO default_loads")
    
#     print("")

#     # Get the records from the table
#     nData = cxn_cursor.execute("SELECT * FROM MEETUP_GUESTS").fetch_pandas_all()
    
#     print("Here is the Snowflake Query ID that handled the above cursor execution")
    print(cxn_cursor.sfqid)
    
    print("")
    
    
except snowflake.connector.errors.ProgrammingError as e:
    # default error message
    print(e)
    # customer error message
    print('Error {0} ({1}): {2} ({3})'.format(e.errno, e.sqlstate, e.msg, e.sfqid))
    
    # ROLLBACK any part of this transaction
    #cxn_cursor.rollback()
    raise e
finally:
    cxn_cursor.close()
    
nData.head(5)

In [117]:
#---------------------------------------
# CLOSE CONNECTION TO SNOWFLAKE
# -- Will require reconnecting/authorizing in the top most cell for connecting
#---------------------------------------
if cxn:
    cxn.close()